# 1. IMPORT LIBRARIES

In [35]:
from datetime import datetime, timedelta, date
import pandas as pd
import pyodbc 

# 2. CONNECT TO SQL SERVER

In [36]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\SQLEXPRESS;"
    "DATABASE=Associate_Rule;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

<>:3: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
<>:3: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
C:\Users\Hiep\AppData\Local\Temp\ipykernel_13068\2065915348.py:3: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
  "SERVER=localhost\SQLEXPRESS;"


In [37]:
df_associated_rule = pd.read_sql_query(''' 
SELECT *
FROM [Associate_Rule].[dbo].[Groceries_dataset]
    ; ''', conn)
df_associated_rule

C:\Users\Hiep\AppData\Local\Temp\ipykernel_13068\1280357271.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_associated_rule = pd.read_sql_query('''


,Member_number,Date,itemDescription,Day,Season,Vacation
0,100000000,2015-07-21,tropical fruit,Sun,Winter,True
1,2552,2015-01-05,whole milk,Mon,Summer,False
2,2300,2015-09-19,pip fruit,Tue,Summer,False
3,-1000000,2015-12-12,other vegetables,Wed,Summer,False
4,3037,2015-02-01,whole milk,Thu,Summer,False
...,...,...,...,...,...,...
38779,4471,2014-10-08,sliced cheese,Sat,Winter,True
38780,2022,2014-02-23,candy,Sun,Winter,True
38781,1097,2014-04-16,cake bar,Mon,Winter,True
38782,1510,2014-12-03,fruit/vegetable juice,Tue,Winter,True


# 3. CLEANING DATA

## 3.1 Remove Duplicates

In [38]:
df_associated_rule = df_associated_rule.drop_duplicates()
df_associated_rule.shape

(38448, 6)

## 3.2 Detect Outliers Using the Interquartile Range (IQR) Method

In [39]:
# Calculate Q1, Q3, and IQR
Q1 = df_associated_rule['Member_number'].quantile(0.25)
Q3 = df_associated_rule['Member_number'].quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers_iqr = df_associated_rule[(df_associated_rule['Member_number'] < lower_bound) | (df_associated_rule['Member_number'] > upper_bound)]
print("Outliers using IQR method:")
print(outliers_iqr)

# Remove outliers
df_associated_rule = df_associated_rule[(df_associated_rule['Member_number'] >= lower_bound) & (df_associated_rule['Member_number'] <= upper_bound)]
print("\nDataFrame after removing outliers (IQR method):")
print(df_associated_rule.shape)

Outliers using IQR method:
   Member_number        Date   itemDescription  Day  Season  Vacation
0      100000000  2015-07-21    tropical fruit  Sun  Winter      True
3       -1000000  2015-12-12  other vegetables  Wed  Summer     False

DataFrame after removing outliers (IQR method):
(38446, 6)


## 3.3 Remove Irrelevant Data

In [40]:
df_associated_rule = df_associated_rule[['Member_number','Date','itemDescription']]
df_associated_rule

,Member_number,Date,itemDescription
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
4,3037,2015-02-01,whole milk
5,4941,2015-02-14,rolls/buns
6,4501,2015-05-08,other vegetables
...,...,...,...
38779,4471,2014-10-08,sliced cheese
38780,2022,2014-02-23,candy
38781,1097,2014-04-16,cake bar
38782,1510,2014-12-03,fruit/vegetable juice


## 3.4 Standardize Capitalization

In [41]:
import re 

def to_snake(col):
    col = col.strip()
    col = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', col) 
    col = col.replace("-", "_")
    col = col.replace(" ", "_")
    col = col.lower()
    return col

df_associated_rule.columns = [to_snake(c) for c in df_associated_rule.columns]

print(df_associated_rule.columns)


Index(['member_number', 'date', 'item_description'], dtype='object')


# 4. PUSH CLEANED DATA TO SQL SEVER

In [42]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\SQLEXPRESS;"
    "DATABASE=Associate_Rule;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "Groceries_cleaned_data"

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_associated_rule[c].dtype)}" for c in df_associated_rule.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_associated_rule.columns))})", df_associated_rule.values.tolist())
conn.commit()

<>:6: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
<>:6: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
C:\Users\Hiep\AppData\Local\Temp\ipykernel_13068\359614989.py:6: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
  "SERVER=localhost\SQLEXPRESS;"


# 5. Create a DataFrame to measure the impact of product pairs when sold together

In [43]:
df_associated_rule

,member_number,date,item_description
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
4,3037,2015-02-01,whole milk
5,4941,2015-02-14,rolls/buns
6,4501,2015-05-08,other vegetables
...,...,...,...
38779,4471,2014-10-08,sliced cheese
38780,2022,2014-02-23,candy
38781,1097,2014-04-16,cake bar
38782,1510,2014-12-03,fruit/vegetable juice


## 1. Create a one-hot encoded transaction table

In [ ]:
# df_associated_rule: member_number, date, item_description
# Mỗi transaction = (member_number, date)
df_transactions = df_associated_rule.drop_duplicates(['member_number','date','item_description'])
df_onehot = df_transactions.pivot_table(index=['member_number','date'],
                                        columns='item_description',
                                        aggfunc=lambda x: 1,
                                        fill_value=0)

# reset index để dễ join
df_onehot.reset_index(inplace=True)


## 2. Calculate the total number of transactions and the support of each product

In [ ]:
total_transactions = df_onehot.shape[0]

# Support của từng sản phẩm
product_support = df_onehot.iloc[:, 2:].sum() / total_transactions  # bỏ 2 cột index


## 3. Create unique product pairs

In [ ]:
from itertools import combinations

products = df_onehot.columns[2:]  # danh sách sản phẩm
pairs = list(combinations(products, 2))
df_pairs = pd.DataFrame(pairs, columns=['product_1','product_2'])
df_pairs['basket'] = df_pairs['product_1'] + ' + ' + df_pairs['product_2']


## 4. Calculate Support Basket, Confidence, and Lift using vectorization


In [ ]:
# Lấy dữ liệu các cột sản phẩm
data_products = df_onehot[products]

# Hàm tính support basket vectorized
support_baskets = []
for p1, p2 in pairs:
    support = ((data_products[p1] & data_products[p2]).sum()) / total_transactions
    support_baskets.append(support)

df_pairs['support_basket'] = support_baskets

# Confidence và Lift
df_pairs['confidence_product_1'] = df_pairs['support_basket'] / df_pairs['product_1'].map(product_support)
df_pairs['confidence_product_2'] = df_pairs['support_basket'] / df_pairs['product_2'].map(product_support)
df_pairs['lift'] = df_pairs['support_basket'] / (df_pairs['product_1'].map(product_support) * df_pairs['product_2'].map(product_support))


## 5. Final DataFrame

In [48]:
# Bước 1: Lấy các cột cần thiết
df_association_rules = df_pairs[['product_1','product_2','basket','support_basket',
                                 'confidence_product_1','confidence_product_2','lift']].copy()

# Bước 2: Chuyển các cột tỷ lệ sang dạng % với 2 chữ số thập phân
for col in ['support_basket','confidence_product_1','confidence_product_2']:
    df_association_rules[col] = df_association_rules[col].apply(lambda x: f"{x*100:.2f}%")

print(df_association_rules.head())



               product_1         product_2  \
0  Instant food products          UHT-milk   
1  Instant food products  abrasive cleaner   
2  Instant food products  artif. sweetener   
3  Instant food products    baby cosmetics   
4  Instant food products              bags   

                                     basket support_basket  \
0          Instant food products + UHT-milk          0.00%   
1  Instant food products + abrasive cleaner          0.00%   
2  Instant food products + artif. sweetener          0.00%   
3    Instant food products + baby cosmetics          0.00%   
4              Instant food products + bags          0.00%   

  confidence_product_1 confidence_product_2  lift  
0                0.00%                0.00%   0.0  
1                0.00%                0.00%   0.0  
2                0.00%                0.00%   0.0  
3                0.00%                0.00%   0.0  
4                0.00%                0.00%   0.0  


## 6. Sort by Lift in descending order and classify the impact level of the two products based on Lift

In [ ]:
# Sort theo lift desc
df_association_rules['lift_float'] = df_pairs['lift']  # lấy giá trị gốc dạng số

df_sorted = df_association_rules.sort_values(by='lift_float', ascending=False).copy()

In [ ]:
#Phân loại mức độ ảnh hưởng của 2 sản phẩm theo Lift
def lift_category(lift):
    if lift > 1.5:
        return 'Strong'
    elif lift > 1.2:
        return 'Moderate'
    elif lift >= 1.0:
        return 'Weak'
    else:
        return 'Negative'

df_sorted['lift_category'] = df_sorted['lift_float'].apply(lift_category)


In [52]:
#Kết quả top 20 cặp mạnh nhất
df_final = df_sorted[['product_1','product_2','basket','support_basket',
                      'confidence_product_1','confidence_product_2','lift','lift_category']]

print(df_final.head(20))


                   product_1          product_2  \
12509  preservation products              soups   
495         artif. sweetener     baby cosmetics   
10057        kitchen utensil              pasta   
12975        rubbing alcohol     sparkling wine   
9345                   honey           prosecco   
13035                    rum             whisky   
5586                cookware             spices   
562         artif. sweetener              honey   
12859         roll products      toilet cleaner   
5986             curd cheese        decalcifier   
6209             decalcifier              soups   
5389       cooking chocolate        light bulbs   
7549          flower (seeds)              honey   
524         artif. sweetener       cocoa drinks   
4685                 cleaner  cooking chocolate   
9377                   honey             spices   
607         artif. sweetener           prosecco   
2184                  brandy        ready soups   
6122             decalcifier   

## 7. Push df_final to SQL Server for analysis

In [53]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\SQLEXPRESS;"
    "DATABASE=Associate_Rule;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "Basket_Analysis"

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_final[c].dtype)}" for c in df_final.columns])
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_final.columns))})", df_final.values.tolist())
conn.commit()

<>:6: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
<>:6: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
C:\Users\Hiep\AppData\Local\Temp\ipykernel_13068\961327586.py:6: SyntaxWarning: "\S" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\S"? A raw string is also an option.
  "SERVER=localhost\SQLEXPRESS;"
